In [41]:
import math
from typing import Sequence, Union, Tuple, Any, Dict, Optional

import clingo
import z3

In [42]:
def solve(programs: Sequence[str], grounding_context=None, sep=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            symbols = model.symbols(atoms=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [43]:
instance = """
object_type_repr(c1, circle, @geometry_id(c1)).
object_type_repr(c2, circle, @geometry_id(c2)).
"""

In [44]:
reasoning = """

region(C) :- object_type_repr(C, circle, _Rep).

extent(A, @extent(R)) :-
  region(A), object_type_repr(A, _T, R).

size(eq, A, A) :- region(A).
size(lt, A, B) :- size(gt, B, A).
size(gt, A, B) :- size(lt, B, A).
size(Cmp, A, B) :-
  region(A), region(B),
  @compare_extents(A, B) = Cmp.

topology(part_of, A, A) :- region(A).
-topology(part_of, A, B) :-
  region(A), region(B),
  not size(eq, A, B), not size(lt, A, B).
-topology(part_of, A, B) :- topology(part_of, B, A), A != B.

topology(part_of, A, B) :-
  region(A), region(B),
  object_type_repr(A, _TA, AR),
  object_type_repr(B, _TB, BR),
  @part_of(AR, BR) = sat.

-topology(part_of, A, B) :-
  region(A), region(B),
  object_type_repr(A, _TA, AR),
  object_type_repr(B, _TB, BR),
  @part_of(AR, BR) = unsat.

"""

In [45]:
class SpatialReasoningContext:

    def __init__(self, geometry_db: Optional[Dict[Union[str, int], Dict[str, Any]]]):
        self.geometry_db = geometry_db
        if self.geometry_db is None:
            self.geometry_db: dict = {}

    def __repr__(self):
        return repr(self.geometry_db)

    def __str__(self):
        return str(self.geometry_db)

    def __get_db_entry(self, representation: clingo.Symbol) -> Tuple[Union[str, int], dict]:
        py_repr = SpatialReasoningContext.get_py_repr(representation)
        if py_repr not in self.geometry_db:
            raise KeyError(
                "{} is not in the geometry_db".format(py_repr))
        return py_repr, self.geometry_db[py_repr]

    def __extent(self, py_repr: Union[str, int]):
        if 'type' not in self.geometry_db[py_repr]:
            raise ValueError('{} does not have a type'.format(py_repr))
        if self.geometry_db[py_repr]['type'] == 'circle':
            self.__extent_circle(py_repr)

    def __extent_circle(self, py_repr: Union[str, int]):
        if 'radius' not in self.geometry_db[py_repr]:
            raise ValueError('{} does not have a radius'.format(py_repr))
        radius = z3.RealVal(self.geometry_db[py_repr]['radius'])
        self.geometry_db[py_repr]['extent'] = z3.RealVal(math.pi) * radius**2


    def geometry_id(self, geometry_object: clingo.Symbol) -> clingo.Symbol:
        py_repr = SpatialReasoningContext.get_py_repr(geometry_object)
        if py_repr not in self.geometry_db:
            self.geometry_db[py_repr] = {}
        assert py_repr in self.geometry_db, "{} should be in geometry_db".format(py_repr)
        return geometry_object

    def extent(self, representation: clingo.Symbol):
        py_repr, entry = self.__get_db_entry(representation)
        if not 'extent' in entry:
            self.__extent(py_repr)
        return clingo.String(str(entry['extent']))

    def compare_extents(self, representation1: clingo.Symbol, representation2: clingo.Symbol):
        self.extent(representation1)
        self.extent(representation2)

        py_repr1, entry1 = self.__get_db_entry(representation1)
        py_repr2, entry2 = self.__get_db_entry(representation2)

        extent1 = entry1['extent']
        extent2 = entry2['extent']
        solver = z3.Solver()
        if solver.check(extent1 == extent2) == z3.sat:
            return clingo.Function('eq')
        elif solver.check(extent1 < extent2) == z3.sat:
            return clingo.Function('lt')
        else:
            assert solver.check(extent1 > extent2) == z3.sat, "Unexpected case: compare_extents({},{})".format(extent1, extent2)
            return clingo.Function('gt')

    def __part_of(self, representation1: clingo.Symbol, representation2: clingo.Symbol):
        eval_sym = SpatialReasoningContext.get_py_repr(clingo.Function('part_of', (representation1, representation2)))
        self.geometry_db[eval_sym] = {}
        py_repr1, entry1 = self.__get_db_entry(representation1)
        py_repr2, entry2 = self.__get_db_entry(representation2)
        if py_repr1 == py_repr2:
            self.geometry_db[eval_sym]['result'] = "sat"
        elif entry1['type'] == 'circle' and entry2['type'] == 'circle':
            inverse_sym = SpatialReasoningContext.get_py_repr(clingo.Function('part_of', (representation2, representation1)))
            if inverse_sym in self.geometry_db:
                if self.geometry_db[inverse_sym]['result'] == 'sat':
                    self.geometry_db[eval_sym]['result'] = 'unsat'

            if 'result' not in self.geometry_db[eval_sym]:
                self.geometry_db[eval_sym]['result'] = str(self.__circle_part_of_circle(py_repr1, py_repr2))

    def __circle_part_of_circle(self, py_repr1: Union[str, int], py_repr2: Union[str, int]) -> z3.CheckSatResult:
        # see https://stackoverflow.com/a/33490701
        solver = z3.Solver()
        x_1 = z3.RealVal(self.geometry_db[py_repr1]['x_coordinate'])
        y_1 = z3.RealVal(self.geometry_db[py_repr1]['y_coordinate'])
        r_1 = z3.RealVal(self.geometry_db[py_repr1]['radius'])
        x_2 = z3.RealVal(self.geometry_db[py_repr2]['x_coordinate'])
        y_2 = z3.RealVal(self.geometry_db[py_repr2]['y_coordinate'])
        r_2 = z3.RealVal(self.geometry_db[py_repr2]['radius'])
        d = z3.Sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2)
        solver.add(r_2 >= (d + r_1))
        return solver.check()


    def part_of(self, representation1: clingo.Symbol, representation2: clingo.Symbol):
        eval_sym = SpatialReasoningContext.get_py_repr(clingo.Function('part_of', (representation1, representation2)))
        if eval_sym not in self.geometry_db:
            self.__part_of(representation1, representation2)
        return clingo.Function(self.geometry_db[eval_sym]['result'])



    @staticmethod
    def get_py_repr(sym: clingo.Symbol):
        if sym.type == clingo.SymbolType.String:
            return sym.string
        elif sym.type == clingo.SymbolType.Number:
            return sym.number

        else:
            return str(sym)


In [46]:
initial_db = dict(
    c1=dict(type='circle', x_coordinate=0.0, y_coordinate=0.0, radius=1.0),
    c2=dict(type='circle', x_coordinate=1.0, y_coordinate=0.0, radius=2.0)
)
spc = SpatialReasoningContext(initial_db)
spc

{'c1': {'type': 'circle', 'x_coordinate': 0.0, 'y_coordinate': 0.0, 'radius': 1.0}, 'c2': {'type': 'circle', 'x_coordinate': 1.0, 'y_coordinate': 0.0, 'radius': 2.0}}

In [47]:
solve([
    reasoning,
    instance
], sep='\n', grounding_context=spc);

Answer 1: {
region(c1)
region(c2)
extent(c1,"3141592653589793/1000000000000000*1**2")
extent(c2,"3141592653589793/1000000000000000*2**2")
object_type_repr(c1,circle,c1)
object_type_repr(c2,circle,c2)
size(eq,c1,c1)
size(eq,c2,c2)
size(gt,c2,c1)
size(lt,c1,c2)
topology(part_of,c1,c1)
topology(part_of,c1,c2)
topology(part_of,c2,c2)
-topology(part_of,c2,c1)
}
SAT 1


In [48]:
spc

{'c1': {'type': 'circle', 'x_coordinate': 0.0, 'y_coordinate': 0.0, 'radius': 1.0, 'extent': 3141592653589793/1000000000000000*1**2}, 'c2': {'type': 'circle', 'x_coordinate': 1.0, 'y_coordinate': 0.0, 'radius': 2.0, 'extent': 3141592653589793/1000000000000000*2**2}, 'part_of(c2,c2)': {'result': 'sat'}, 'part_of(c1,c2)': {'result': 'sat'}, 'part_of(c2,c1)': {'result': 'unsat'}, 'part_of(c1,c1)': {'result': 'sat'}}